In [1]:
from pathlib import Path
from cpsat_autotune import (
    tune_for_quality_within_timelimit,
    import_model,
    tune_time_to_optimal,
)

/Users/krupke/anaconda3/envs/mo312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = import_model(Path("models/medium_hg.pb"))

In [3]:
# tune_for_quality_within_timelimit(models[0], 3, 10, "minimize", n_samples_per_param=2, max_samples_per_param=5, n_trials=100)
tune_time_to_optimal(
    model,
    timelimit_in_s=6,
    n_samples_per_param=5,
    max_samples_per_param=10,
    n_trials=50,
)

[I 2024-08-14 17:19:35,959] A new study created in memory with name: no-name-651dc3b6-b8b0-4d30-8894-4acaa7b64077


Computing baseline


[I 2024-08-14 17:20:30,835] Trial 0 finished with value: -5.487255 and parameters: {'repair_hint': False, 'use_lb_relax_lns': False, 'preferred_variable_order': 0, 'use_erwa_heuristic': False, 'linearization_level': 1, 'fp_rounding': 3, 'randomize_search': False, 'diversify_lns_params': False, 'add_objective_cut': False, 'use_objective_lb_search': False, 'use_objective_shaving_search': False, 'search_branching': 0, 'cut_level': 1, 'max_all_diff_cut_size': 1, 'symmetry_level': 2, 'max_presolve_iterations': 2, 'cp_model_probing_level': 2, 'presolve_probing_deterministic_time_limit': 4, 'presolve_bve_threshold': 1, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': False, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': False, 'ignore_subsolvers:reduced_costs': False, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': False, 'ignore_subsolvers:lb_tree_search': False, 'ignore_subsolvers:pro

Baseline: [-5.294558, -5.420106, -5.443795, -5.369062, -5.444739, -5.822297, -5.740075, -5.345089, -5.38167, -5.611159]


[I 2024-08-14 17:20:38,154] Trial 1 finished with value: -60.0 and parameters: {'repair_hint': True, 'use_lb_relax_lns': True, 'preferred_variable_order': 0, 'use_erwa_heuristic': True, 'linearization_level': 1, 'fp_rounding': 2, 'randomize_search': False, 'diversify_lns_params': False, 'add_objective_cut': True, 'use_objective_lb_search': False, 'use_objective_shaving_search': True, 'search_branching': 8, 'cut_level': 1, 'max_all_diff_cut_size': 1, 'symmetry_level': 2, 'max_presolve_iterations': 3, 'cp_model_probing_level': 0, 'presolve_probing_deterministic_time_limit': 1, 'presolve_bve_threshold': 1, 'ignore_subsolvers:default_lp': False, 'ignore_subsolvers:fixed': True, 'ignore_subsolvers:no_lp': False, 'ignore_subsolvers:max_lp': False, 'ignore_subsolvers:pseudo_costs': True, 'ignore_subsolvers:reduced_costs': True, 'ignore_subsolvers:quick_restart': False, 'ignore_subsolvers:quick_restart_no_lp': True, 'ignore_subsolvers:lb_tree_search': True, 'ignore_subsolvers:probing': False}.

------------------------------------------------------------
Parameters:
	cp_model_probing_level: 1
	use_lb_relax_lns: True
	max_presolve_iterations: 1
	symmetry_level: 0
	ignore_subsolvers: ('fixed', 'no_lp', 'probing', 'quick_restart_no_lp')
	linearization_level: 2
	search_branching: 1
	randomize_search: True
	add_objective_cut: True
	diversify_lns_params: True
	preferred_variable_order: 1
	repair_hint: True
	max_all_diff_cut_size: 32
	use_erwa_heuristic: True
Difference to default: 3.2086112
This is significant: True
------------------------------------------------------------
------------------------------------------------------------
Parameters:
	presolve_probing_deterministic_time_limit: 10.0
	randomize_search: True
	add_objective_cut: True
	preferred_variable_order: 2
	diversify_lns_params: True
	max_all_diff_cut_size: 128
	symmetry_level: 0
Difference to default: 0.39306280000000005
This is significant: True
------------------------------------------------------------
--------